In [1]:
using DataFrames, CSV
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots, GRUtils
using Distributions
using Gurobi, JuMP

In [2]:
# Constants
N = 3 ; # number of passengers
K = 3 ;  # number of shuttles

In [3]:
# Use this cost later to load costs from BingsMap. For now, we will simply use the euclidian distance as shown below.
c = Matrix(DataFrame(CSV.File("../SimulatedCosts.csv")))
c = c[:, 2:size(c)[2]];

t = c  #let's consider in this initial model that the travel cost and the time and equivalent. 

8×8 Matrix{Float64}:
 1.2  4.0  1.2  4.5  1.2  0.9  1.2   0.9
 3.0  3.0  3.2  1.4  1.6  0.6  0.7   0.5
 1.4  5.0  4.6  1.9  3.8  3.0  0.4   0.2
 1.9  2.0  1.9  2.5  2.5  5.0  1.8   1.5
 2.5  1.0  2.4  1.2  6.2  2.0  1.3   1.7
 1.2  5.0  1.9  4.0  4.5  1.2  0.9   1.2
 4.0  2.0  2.5  2.0  2.1  2.4  6.5  18.3
 2.0  1.0  1.2  3.0  3.0  5.0  4.6   1.9

In [4]:
# Data
data = Matrix(DataFrame(CSV.File("../SimulatedDataTest.csv")));

q = data[1:2*N+2, 4];
d = data[1:2*N+2, 5];
e = data[1:2*N+2, 7];
l = data[1:2*N+2, 8];

#compute costs/time as euclidian distance.
c = zeros(2 * N + 2, 2 * N + 2)
for i = 1:2*N+2
    for j = 1:2*N+2
        c[i, j] = norm(data[i, 2:3] - data[j, 2:3]) / 10
    end
end

t = c  #let's consider in this initial model that the travel cost and the time and equivalent. 

8×8 Matrix{Float64}:
 0.0       0.291247  0.150659  0.743121  …  0.693661  1.31528   0.0
 0.291247  0.0       0.326118  0.620831     0.488037  1.03742   0.291247
 0.150659  0.326118  0.0       0.869122     0.789801  1.28146   0.150659
 0.743121  0.620831  0.869122  0.0          0.238253  1.26881   0.743121
 0.668374  0.379722  0.697821  0.585946     0.353577  0.733003  0.668374
 0.693661  0.488037  0.789801  0.238253  …  0.0       1.03335   0.693661
 1.31528   1.03742   1.28146   1.26881      1.03335   0.0       1.31528
 0.0       0.291247  0.150659  0.743121     0.693661  1.31528   0.0

### The model

In [5]:
model = Model(Gurobi.Optimizer)

V = [1, 2, 3, 4, 5, 6, 7, 8]   # all vertices
P = [2, 3, 4]             # pick up vertices
D = [5, 6, 7]             # drop off vertices
PUD = [2, 3, 4, 5, 6, 7]     # all pick up and drop off vertices together (all vertices except depot)

T = [100, 100, 100]       # maximum duration of route k
L = 100                  # maximum time of a drive
Q = [3, 3, 3]             # capacity of each car

# Variables
@variable(model, x[1:2*N+2, 1:2*N+2, 1:K] >= 0, Bin)   # x[i, j, k] = 1 if we go from node i to node j with vehicle k.
@variable(model, w[1:2*N+2, 1:K] >= 0, Int)            # w[i,k] load of vehicle k when arriving upon leaving vertex i
@variable(model, r[1:N, 1:K] >= 0)                          # r[i,k] travel time of passenger i (Note that passenger i is identified by) the vertex i+1

@variable(model, u[1:2*N+2, 1:K] >= 0)                 # u[i,k] time as which the vertex i is served by vehicle k
# @variable(model, u_depot[1:2, 1:K] >= 0)
# @variable(model, u[1:2*N] >= 0)
# @variable(model, u[[1:K],1:2*N,[1:K]] >= 0) #------< MAKE THAT MORE ELEGANT WHEN I WILL HAVE INTERNET.


#############  Constraints  #############

# only one car can bring passenger {i, i+N} to its final destination.
@constraint(model, [i in P], sum(sum(x[i, j, k] for j in V) for k = 1:K) == 1)

# each vehicle starts at the depot
@constraint(model, [k = 1:K], sum(x[1, j, k] for j in V) == 1)

# each vehicle ends at the depot
@constraint(model, [k = 1:K], sum(x[i, 2*N+2, k] for i in V) == 1)

# each passenger is picked and droped by the same car
@constraint(model, [i in P, k = 1:K], sum(x[i, j, k] for j in V) - sum(x[i+N, j, k] for j in V) == 0)

# vehicle k follows the edges sequentially
@constraint(model, [i in PUD, k = 1:K], sum(x[j, i, k] for j in V) - sum(x[i, j, k] for j in V) == 0)

#### -------- CORRETIONS ######

# the time served of vertex i
@constraint(model, [i in V, j in V, k = 1:K], u[j, k] >= (u[i, k] + d[i] + t[i, j]) * x[i, j, k])

# respecting travel window of passenger i
@constraint(model, [i in V, k = 1:K], e[i] <= u[i,k])
@constraint(model, [i in V, k = 1:K], u[i,k] <= l[i])

# PREVIOUS updating the travel time of passenger i
@constraint(model, [i in P, k = 1:K], r[i-1,k] == u[N+i, k] - (u[i, k] + d[i]))

# respecting travel window of passenger i
@constraint(model, [i in P], t[i, N+i] <= r[i-1])

# respecting each car's capacity
@constraint(model, [i in V, k = 1:K], max(0, q[i]) <= w[i, k])
@constraint(model, [i in V, k = 1:K], w[i, k] <= min(Q[k], Q[k] + q[i]))

# updating the load of the vehicle
@constraint(model, [i in V, j in V, k = 1:K], w[j, k] >= (w[i, k] + q[j]) * x[i, j, k])

# ---> Let's try to fix the load problem: the load was not null for the useless cars. This was fixed.
@constraint(model, [i in V, k = 1:K], sum(x[:,i,k])*q[i] <= w[i, k])
@constraint(model, [i in V, k = 1:K], 0 <= w[i, k])

@constraint(model, [i in V, k = 1:K], w[i, k] <= sum(x[:,i,k])*Q[k])
@constraint(model, [i in V, k = 1:K], w[i, k] <= Q[k] + sum(x[:,i,k])*q[i])

### --- ADDITIONAL CONSTRAINTS -----

# Pick up before drop off
@constraint(model, [i in P], u[i-1] <= u[N+i-1])

# # No passengers at the beginning / end
# @constraint(model, [k = 1:K], w[1, k] == 0)
# @constraint(model, [k = 1:K], w[2*N+2, k] == 0)

# objective
@objective(model, Min, sum(sum(sum(c[i, j] * x[i, j, k] for k = 1:K) for i in V) for j in V));

# Objective with a penalty function in the ride time and on the load of each car
# @objective(model, Min, sum(sum(sum(c[i, j] * x[i, j, k] for k = 1:K) for i in V) for j in V) + sum(w) + sum(u) + sum(u_depot) + sum(x));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


In [6]:
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 243 rows, 249 columns and 1224 nonzeros
Model fingerprint: 0xcdd3400f
Model has 384 quadratic constraints
Variable types: 33 continuous, 216 integer (192 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [2e-01, 3e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-01, 1e+03]
Presolve added 64 rows and 0 columns
Presolve removed 0 rows and 69 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 144, time in user-callback 0.00 sec
